In [17]:
from db_operations import Db_Opeartions as db_op
from data_pre_process import Data_Pre_Process as dpre
import pickle
import os
import pandas as pd
import numpy as np

#df = pd.read_csv("Data/TRAINED.csv")
#X = df




class Predict_Class:
    
    
    def __init__(self,file_obj,logger_obj):
        self.file_obj = file_obj
        self.logger_obj = logger_obj
    def predict(self,data):
        
        
        #Preprocessing new data
        dat_pre = dpre(file_obj,logger_obj)
        #Cleaning spaces if any
        df = dat_pre.space_cleaning(data)
        #Removing irrelevant Columns
        df = dat_pre.nouse_columns_remove(df)
        #Splits data to dependent and independent 
        dfx = dat_pre.split_dependent_independent(df)
        #Checks for null values and if any columns which has nul value returns
        cols, null_col_df = dat_pre.checks_for_null(dfx)
        #Seperates data to categorical and numerical
        df_cat, df_num = dat_pre.seperate_cat_num(dfx)
        cols = df_num.columns
        df_cat[cols] = df_num[cols].values
        #Cleaning categorical data
        df_cat = dat_pre.clean_cat_data(df_cat)
        #Creates one hot encoding object-dictionary for encoding nominal values
        oh_dict = dat_pre.create_one_hot1()
        #Performs One hot encoding on nominal values
        df_enc = dat_pre.oh_encode1(oh_dict,df_cat)
        #Performs frequency encoding on nominal variables which has more unique values in number
        df_fr_enc = dat_pre.freq_encode(df_enc)
        #Missing numerical values imputation
        df_cat_num = dat_pre.impute_missing_num_data(df_fr_enc)
        #Scaling the data
        df_xcn = dat_pre.scales_data(df_cat_num)
        #Identifies the cluster in which the given data belongs to
        print("Going to clusterr.......")
        
        print(df_xcn)
        #Kmeans = open("models/Kmeans.pkl",'rb')
        #Kms = pickle.load(Kmeans)
        #Kmeans.close()
        #cluster = Kms.predict(df_xcn)
        #print("Cluster is ", cluster)
        #Finds the model to use based on the cluster
        file_list = os.listdir('models/')
        filename = self.get_fn(file_list)
        #Loading the model
        model_file = open("models/"+filename,'rb')
        model = pickle.load(model_file)
        model_file.close()
        #Predicting 
        predicted_value = model.predict(df_xcn)
        df_xcn["Y"] = predicted_value
        #Connects with the database
        #db_op = db_ops(self.file_obj, self.logger_obj)
        #session = db_op.connect_db()
        #Inserting the whole data with prediction to database
        #db_op.db_input_data(df_xcn,session)

        return predicted_value

    ##Local function to find the model file to load
    def get_fn(self,file_list):
        for i in file_list:
            if ".pkl" in i:
                 return i
    

In [18]:
import pandas as pd
import numpy as np

df = pd.read_csv("UCI_Credit_Card.csv")
X = df.drop(labels="default.payment.next.month", axis=1)
Y = df["default.payment.next.month"]



In [19]:
x = df.sample(1)
x

,id,limit_bal,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
14424,14425,210000.0,1,1,2,27,0,0,0,2,...,120763.0,118722.0,121249.0,5650.0,10100.0,4400.0,0.0,4500.0,4500.0,0


In [20]:
import pandas as pd
data = [[0,45622,2,1,2,42,2,-1,3,-1,3,1,456,5236,145,256,456,756,1200,6000,200,200,500,500]]

dat = pd.DataFrame(data, columns=['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6'])

In [21]:
#len(data[0,45622,2,1,2,42,2,-1,3,-1,3,1,456,5236,145,256,456,756,1200,6000,200,200,500,500])
len(data[0])

24

In [22]:
from application_logging import logger as logr
file_obj = open("Predict_logs.txt","a+")
logger_obj = logr.App_Logger()
pc = Predict_Class(file_obj,logger_obj)

In [23]:
prblm = pc.predict(dat)

RangeIndex(start=0, stop=1, step=1)
['SEX_1']
[[1.]]
RangeIndex(start=0, stop=2, step=1)
['MARRIAGE_1', 'MARRIAGE_2']
[[1. 0.]]
RangeIndex(start=0, stop=3, step=1)
['EDUCATION_1', 'EDUCATION_2', 'EDUCATION_3']
[[0. 0. 0.]]
Going to clusterr.......
   PAY_0   PAY_2   PAY_3   PAY_4   PAY_5  PAY_6  LIMIT_BAL  AGE  BILL_AMT1  \
0  0.091  0.2019  0.0084  0.1906  0.0065    0.0      45622   42        456   

   BILL_AMT2  ...  PAY_AMT3  PAY_AMT4  PAY_AMT5  PAY_AMT6  SEX_1  MARRIAGE_1  \
0       5236  ...       200       200       500       500    1.0         1.0   

   MARRIAGE_2  EDUCATION_1  EDUCATION_2  EDUCATION_3  
0         0.0          0.0          0.0          0.0  

[1 rows x 26 columns]


C:\Users\USER\anaconda3\lib\site-packages\sklearn\utils\extmath.py:847: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
C:\Users\USER\anaconda3\lib\site-packages\sklearn\utils\extmath.py:689: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = op(x, *args, **kwargs)


In [20]:
df = pd.read_csv("UCI_Credit_Card.csv")

In [21]:
df.columns

Index(['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'default.payment.next.month'],
      dtype='object')

In [24]:
prblm

array([1], dtype=int64)

In [64]:
import numpy as np
np.nan_to_num(prblm[prblm.isnull().values])

array([[9.1000e-02, 2.0190e-01, 8.4000e-03, 1.9060e-01, 6.5000e-03,
        0.0000e+00, 4.5622e+04, 4.2000e+01, 4.5600e+02, 5.2360e+03,
        1.4500e+02, 2.5600e+02, 4.5600e+02, 7.5600e+02, 1.2000e+03,
        6.0000e+03, 2.0000e+02, 2.0000e+02, 5.0000e+02, 5.0000e+02,
        1.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00]])

In [69]:
prblm.replace(np.nan, 0)

,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,LIMIT_BAL,AGE,BILL_AMT1,BILL_AMT2,...,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,SEX_1,MARRIAGE_1,MARRIAGE_2,EDUCATION_1,EDUCATION_2,EDUCATION_3
0,0.091,0.2019,0.0084,0.1906,0.0065,0.0,45622,42,456,5236,...,200,200,500,500,1.0,1.0,0.0,0.0,0.0,0.0


In [72]:
prblm.replace([np.inf, -np.inf, np.nan],0)

,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,LIMIT_BAL,AGE,BILL_AMT1,BILL_AMT2,...,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,SEX_1,MARRIAGE_1,MARRIAGE_2,EDUCATION_1,EDUCATION_2,EDUCATION_3
0,0.091,0.2019,0.0084,0.1906,0.0065,0.0,45622,42,456,5236,...,200,200,500,500,1.0,1.0,0.0,0.0,0.0,0.0


In [8]:
prblm

array([1], dtype=int64)

In [10]:
x = [[np.nan,5,10,6,759,np.inf]]

In [11]:
x = pd.DataFrame(x)

In [12]:
x

,0,1,2,3,4,5
0,NaN,5,10,6,759,inf


In [15]:
x.replace(np.inf, 0 , inplace=True)

In [16]:
x

,0,1,2,3,4,5
0,0.0,5,10,6,759,0.0
